In [1]:
import os
import json
import re
from dotenv import load_dotenv
from openai import AzureOpenAI, OpenAIError

load_dotenv()  # .env 파일에 있는 환경변수 로드

aoi_api_key = os.getenv("AZURE_OPENAI_API_KEY")
aoi_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
aoi_gen_model = os.getenv("AZURE_GENERATION_MODEL")
aoi_version = os.getenv("AZURE_GENERATION_MODEL_VERSION")
client = AzureOpenAI(azure_endpoint=aoi_endpoint,api_key=aoi_api_key, api_version=aoi_version)


In [6]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.agents import Tool,  create_react_agent
from openai import AzureOpenAI, OpenAIError


In [ ]:
from langchain_openai import AzureChatOpenAI

def get_current_time(*args, **kwargs):
    """Get the current time."""
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")


tools= [
    Tool(
        name="get_current_time",
        func=get_current_time,
        description="Useful for when you need to get the current time."
    ),
]

# LLM with function call
llm = AzureChatOpenAI(
    azure_endpoint=aoi_endpoint,
    api_key=aoi_api_key, 
    api_version=aoi_version,
    deployment_name=aoi_gen_model
)
prompt = hub.pull("hwchase17/react")

In [14]:
agent = create_react_agent(llm, tools=tools,prompt=prompt,stop_sequence=True)

In [17]:
agent_excutor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [16]:
reponse = agent_excutor.invoke({"input": "What is the current time?"})
print(reponse['output'])



> Entering new AgentExecutor chain...
I need to determine the current time.
Action: get_current_time
Action Input: None2025-05-07 12:22:00I now know the final answer.
Final Answer: The current time is 2025-05-07 12:22:00.

> Finished chain.
The current time is 2025-05-07 12:22:00.


REact 방식으로 작성함
prompt :"PromptTemplate(
    input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'],  # 템플릿에 들어갈 변수들
    input_types={},  # 각 변수의 타입을 지정 (여기서는 비어 있음)
    partial_variables={},  # 템플릿의 일부 변수를 미리 채워둘 때 사용 (여기서는 비어 있음)
    metadata={  # 템플릿에 대한 메타데이터
        'lc_hub_owner': 'hwchase17',  # LangChain Hub의 소유자 (LangChain 개발자)
        'lc_hub_repo': 'react',      # LangChain Hub의 레포지토리 이름 (ReAct 에이전트 관련)
        'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'  # 특정 커밋 해시
    },
    template='''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''  # 실제 LLM에게 전달될 프롬프트의 구조
)"